In [ ]:
# 单个看板
import pandas as pd
from pyecharts.charts import Bar, Line, Pie, Scatter, Page
from pyecharts import options as opts

# 加载和合并数据集
submit_records = pd.read_csv('./数据/Data_SubmitRecord/SubmitRecord-Class1.csv')
question_attributes = pd.read_csv('./数据/题目属性.csv')
student_attributes = pd.read_csv('./数据/学生属性.csv')

# 合并数据
merged_data = pd.merge(submit_records, question_attributes, on='title_ID', how='left')
merged_data = pd.merge(merged_data, student_attributes, on='student_ID', how='left')

# 加载数据
file_path = './数据/合并后的数据.xlsx'
data = pd.read_excel(file_path)

# 确保 timeconsume 列为数值，无法转换的设置为 NaN
data['timeconsume'] = pd.to_numeric(data['timeconsume'], errors='coerce')

# 1. 编程语言使用偏好分析
language_distribution = data['method'].value_counts().reset_index()
language_distribution.columns = ['编程语言', '答题次数']

pie_language = (
    Pie()
    .add(
        "编程语言",
        [list(z) for z in zip(language_distribution['编程语言'], language_distribution['答题次数'])]
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="编程语言使用偏好"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%")
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}次"))
)

# 2. 编程语言与平均得分分析
language_score = data.groupby('method')['score_x'].mean().reset_index()
language_score.columns = ['编程语言', '平均得分']

bar_language_score = (
    Bar()
    .add_xaxis(language_score['编程语言'].tolist())
    .add_yaxis("平均得分", language_score['平均得分'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="编程语言与平均得分"),
        xaxis_opts=opts.AxisOpts(name="编程语言"),
        yaxis_opts=opts.AxisOpts(name="平均得分", min_=0, max_=4)
    )
)

# 3. 编程语言与答题耗时分析
language_timeconsume = data.dropna(subset=['timeconsume']).groupby('method')['timeconsume'].mean().reset_index()
language_timeconsume.columns = ['编程语言', '平均答题耗时']

line_language_timeconsume = (
    Line()
    .add_xaxis(language_timeconsume['编程语言'].tolist())
    .add_yaxis("平均耗时 (秒)", language_timeconsume['平均答题耗时'].tolist(), label_opts=opts.LabelOpts(formatter="{c} s"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="编程语言与答题耗时"),
        xaxis_opts=opts.AxisOpts(name="编程语言"),
        yaxis_opts=opts.AxisOpts(name="平均耗时 (秒)")
    )
)

# 4. 知识点掌握情况分析
knowledge_mastery = data.groupby('knowledge')['score_x'].mean().reset_index()
knowledge_mastery.columns = ['知识点', '平均得分']

bar_knowledge = (
    Bar()
    .add_xaxis(knowledge_mastery['知识点'].tolist())
    .add_yaxis("平均得分", knowledge_mastery['平均得分'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="知识点掌握情况"),
        xaxis_opts=opts.AxisOpts(name="知识点"),
        yaxis_opts=opts.AxisOpts(name="平均得分")
    )
)

# 5. 答题活跃时段分析
data['time'] = pd.to_datetime(data['time'], unit='s')
data['hour'] = data['time'].dt.hour
hourly_distribution = data.groupby('hour').size().reset_index(name='答题次数')

bar_time = (
    Bar()
    .add_xaxis([f"{i}点" for i in hourly_distribution['hour']])
    .add_yaxis("答题次数", hourly_distribution['答题次数'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="答题活跃时段分布"),
        xaxis_opts=opts.AxisOpts(name="时段"),
        yaxis_opts=opts.AxisOpts(name="答题次数")
    )
)

# 6. 偏好题型分析
question_type_distribution = data['knowledge'].value_counts().reset_index()
question_type_distribution.columns = ['题型', '答题次数']

bar_type = (
    Bar()
    .add_xaxis(question_type_distribution['题型'].tolist())
    .add_yaxis("答题次数", question_type_distribution['答题次数'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="题型偏好分析"),
        xaxis_opts=opts.AxisOpts(name="题型"),
        yaxis_opts=opts.AxisOpts(name="答题次数")
    )
)

# 7. 正确答题率分析
correct_rate = data.groupby('knowledge')['state'].apply(
    lambda x: (x == 'Absolutely_Correct').mean()
).reset_index()
correct_rate.columns = ['题型', '正确答题率']

line_correct_rate = (
    Line()
    .add_xaxis(correct_rate['题型'].tolist())
    .add_yaxis("正确答题率", (correct_rate['正确答题率'] * 100).tolist(), label_opts=opts.LabelOpts(formatter="{c}%"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="正确答题率分析"),
        xaxis_opts=opts.AxisOpts(name="题型"),
        yaxis_opts=opts.AxisOpts(name="正确答题率 (%)")
    )
)

# 8. 学习模式与知识掌握关系 - 答题耗时分析
learning_behavior = data.groupby('timeconsume')['score_x'].mean().reset_index()
learning_behavior.columns = ['答题耗时', '平均得分']

line_timeconsume = (
    Line()
    .add_xaxis(learning_behavior['答题耗时'].tolist())
    .add_yaxis("平均得分", learning_behavior['平均得分'].tolist(), is_smooth=True)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="答题耗时与知识掌握程度关系"),
        xaxis_opts=opts.AxisOpts(name="答题耗时 (秒)"),
        yaxis_opts=opts.AxisOpts(name="平均得分")
    )
)

# 9. 每道题的得分与答题量情况（气泡图）
title_score_data = data.groupby('title_ID').agg(
    avg_score=('score_x', 'mean'),
    count=('score_x', 'size')
).reset_index()

title_list = title_score_data['title_ID'].tolist()
avg_score_list = title_score_data['avg_score'].tolist()
count_list = title_score_data['count'].tolist()

scatter = (
    Scatter()
    .add_xaxis(title_list)
    .add_yaxis(
        "Average Score",
        avg_score_list,
        symbol_size=[min(50, max(10, c / 2)) for c in count_list],  # 控制气泡大小范围
        itemstyle_opts=opts.ItemStyleOpts(opacity=0.7)  # 设置气泡透明度
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="每道题的得分与答题量关系"),
        xaxis_opts=opts.AxisOpts(name="题目ID", axislabel_opts={"rotate": 45}),
        yaxis_opts=opts.AxisOpts(name="平均得分", min_=0, max_=4),  # 假设得分满分为4
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{b}: 平均得分 {c}"),
        legend_opts=opts.LegendOpts(pos_top="5%", pos_right="5%"),  # 设置图例位置为右上角
        visualmap_opts=opts.VisualMapOpts(
            type_="size",
            max_=50,
            min_=10,
            dimension=1,
            pos_right="5%",  # 右对齐
            pos_top="15%",   # 图例下方
        )
    )
)

# 10. 年龄与答题表现分析
age_score = data.groupby('age')['score_x'].mean().reset_index()
age_score.columns = ['年龄', '平均得分']

bar_age_score = (
    Bar()
    .add_xaxis(age_score['年龄'].astype(str).tolist())
    .add_yaxis("平均得分", age_score['平均得分'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="年龄与答题表现"),
        xaxis_opts=opts.AxisOpts(name="年龄"),
        yaxis_opts=opts.AxisOpts(name="平均得分")
    )
)

# 生成包含所有图表的HTML看板
page = Page(layout=Page.DraggablePageLayout)
page.add(
    pie_language, 
    bar_language_score, 
    line_language_timeconsume, 
    bar_knowledge, 
    bar_time, 
    bar_type, 
    line_correct_rate, 
    line_timeconsume, 
    scatter, 
    bar_age_score
)

# 渲染为 HTML 文件
page.render("单个班级看板.html")


In [ ]:
from pyecharts.charts import Bar, Page, Tab
from pyecharts import options as opts
import pandas as pd

# 加载数据
file_path = './数据/合并后的数据.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# 获取所有题目 ID 列表
titles = data['title_ID'].unique()

# 创建一个 Tab 容器
tab = Tab()

# 遍历每个题目 ID，生成其得分分布直方图，并将其加入 Tab 中
for title in titles:
    # 筛选出当前题目的数据
    title_data = data[data['title_ID'] == title]
    
    # 统计当前题目的得分分布
    score_distribution = title_data['score_x'].value_counts().sort_index()
    score_x = score_distribution.index.tolist()
    score_y = score_distribution.values.tolist()
    
    # 创建直方图
    bar = (
        Bar()
        .add_xaxis(score_x)
        .add_yaxis("答题次数", score_y)
        .set_global_opts(
            title_opts=opts.TitleOpts(title=f"题目 {title} 的得分分布"),
            xaxis_opts=opts.AxisOpts(name="得分"),
            yaxis_opts=opts.AxisOpts(name="次数"),
        )
    )
    
    # 将每个题目的图表添加到 Tab 容器中，并以题目 ID 作为 Tab 名称
    tab.add(bar, f"题目 {title}")

# 保存成 HTML 文件，包含所有题目的得分分布图表
tab.render("每个题目得分直方图.html")

In [19]:
# 导入必要的库
import pandas as pd
from pyecharts.charts import Bar, Line, Pie, Scatter, Page
from pyecharts import options as opts
import os

# 加载题目和学生的属性数据
question_attributes = pd.read_csv('./数据/题目属性.csv')
student_attributes = pd.read_csv('./数据/学生属性.csv')

# 获取所有班级数据文件的路径
file_paths = [
    './数据/Data_SubmitRecord/SubmitRecord-Class1.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class2.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class3.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class4.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class5.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class6.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class7.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class8.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class9.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class10.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class11.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class12.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class13.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class14.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class15.csv'
]

# 遍历每个班级的数据文件，生成独立的 HTML 看板页面
for file_path in file_paths:
    submit_records = pd.read_csv(file_path)
    class_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # 合并数据
    merged_data = pd.merge(submit_records, question_attributes, on='title_ID', how='left')
    merged_data = pd.merge(merged_data, student_attributes, on='student_ID', how='left')
    merged_data['timeconsume'] = pd.to_numeric(merged_data['timeconsume'], errors='coerce')

    # 定义所有图表
    language_distribution = merged_data['method'].value_counts().reset_index()
    language_distribution.columns = ['编程语言', '答题次数']
    pie_language = (
        Pie()
        .add("编程语言", [list(z) for z in zip(language_distribution['编程语言'], language_distribution['答题次数'])])
        .set_global_opts(title_opts=opts.TitleOpts(title="编程语言使用偏好"))
    )

    language_score = merged_data.groupby('method')['score_x'].mean().reset_index()
    bar_language_score = (
        Bar()
        .add_xaxis(language_score['method'].tolist())
        .add_yaxis("平均得分", language_score['score_x'].tolist())
        .set_global_opts(title_opts=opts.TitleOpts(title="编程语言与平均得分"))
    )

    language_timeconsume = merged_data.dropna(subset=['timeconsume']).groupby('method')['timeconsume'].mean().reset_index()
    line_language_timeconsume = (
        Line()
        .add_xaxis(language_timeconsume['method'].tolist())
        .add_yaxis("平均耗时 (秒)", language_timeconsume['timeconsume'].tolist())
        .set_global_opts(title_opts=opts.TitleOpts(title="编程语言与答题耗时"))
    )

    knowledge_mastery = merged_data.groupby('knowledge')['score_x'].mean().reset_index()
    bar_knowledge = (
        Bar()
        .add_xaxis(knowledge_mastery['knowledge'].tolist())
        .add_yaxis("平均得分", knowledge_mastery['score_x'].tolist())
        .set_global_opts(title_opts=opts.TitleOpts(title="知识点掌握情况"))
    )

    merged_data['hour'] = pd.to_datetime(merged_data['time'], unit='s').dt.hour
    hourly_distribution = merged_data.groupby('hour').size().reset_index(name='答题次数')
    bar_time = (
        Bar()
        .add_xaxis([f"{i}点" for i in hourly_distribution['hour']])
        .add_yaxis("答题次数", hourly_distribution['答题次数'].tolist())
        .set_global_opts(title_opts=opts.TitleOpts(title="答题活跃时段分布"))
    )

    # 6. 偏好题型分析
    question_type_distribution = merged_data['knowledge'].value_counts().reset_index()
    question_type_distribution.columns = ['题型', '答题次数']

    bar_type = (
        Bar()
        .add_xaxis(question_type_distribution['题型'].tolist())
        .add_yaxis("答题次数", question_type_distribution['答题次数'].tolist())
        .set_global_opts(
            title_opts=opts.TitleOpts(title="题型偏好分析"),
            xaxis_opts=opts.AxisOpts(name="题型"),
            yaxis_opts=opts.AxisOpts(name="答题次数")
        )
    )

    correct_rate = merged_data.groupby('knowledge')['state'].apply(
        lambda x: (x == 'Absolutely_Correct').mean()
    ).reset_index()
    line_correct_rate = (
        Line()
        .add_xaxis(correct_rate['knowledge'].tolist())
        .add_yaxis("正确答题率", (correct_rate['state'] * 100).tolist())
        .set_global_opts(title_opts=opts.TitleOpts(title="正确答题率分析"))
    )

    learning_behavior = merged_data.groupby('timeconsume')['score_x'].mean().reset_index()
    line_timeconsume = (
        Line()
        .add_xaxis(learning_behavior['timeconsume'].tolist())
        .add_yaxis("平均得分", learning_behavior['score_x'].tolist())
        .set_global_opts(title_opts=opts.TitleOpts(title="答题耗时与知识掌握程度关系"))
    )

    # 9. 每道题的得分与答题量关系（气泡图）
    title_score_data = merged_data.groupby('title_ID').agg(
        avg_score=('score_x', 'mean'),
        count=('score_x', 'size')
    ).reset_index()

    title_list = title_score_data['title_ID'].tolist()
    avg_score_list = title_score_data['avg_score'].tolist()
    count_list = title_score_data['count'].tolist()

    scatter = (
        Scatter()
        .add_xaxis(title_list)
        .add_yaxis(
            "Average Score",
            avg_score_list,
            symbol_size=[min(50, max(10, c / 2)) for c in count_list],  # 根据答题量调整气泡大小
            itemstyle_opts=opts.ItemStyleOpts(opacity=0.7)  # 设置气泡透明度
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="每道题的得分与答题量关系"),
            xaxis_opts=opts.AxisOpts(name="题目ID", axislabel_opts={"rotate": 45}),
            yaxis_opts=opts.AxisOpts(name="平均得分", min_=0, max_=4),  # 假设得分满分为4
            tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{b}: 平均得分 {c}"),
            legend_opts=opts.LegendOpts(pos_top="5%", pos_right="5%"),  # 设置图例位置为右上角
            visualmap_opts=opts.VisualMapOpts(
                type_="size",
                max_=50,
                min_=10,
                dimension=1,
                pos_right="5%",  # 右对齐
                pos_top="15%",   # 图例下方
            )
        )
    )


    age_score = merged_data.groupby('age')['score_x'].mean().reset_index()
    bar_age_score = (
        Bar()
        .add_xaxis(age_score['age'].astype(str).tolist())
        .add_yaxis("平均得分", age_score['score_x'].tolist())
        .set_global_opts(title_opts=opts.TitleOpts(title="年龄与答题表现"))
    )

    # 加入到页面布局中
    charts = [
        pie_language, bar_language_score, line_language_timeconsume, bar_knowledge,
        bar_time, bar_type, line_correct_rate, line_timeconsume, scatter, bar_age_score
    ]
    
    # 创建 HTML 页面，并在表格布局中两两排列图表
    with open(f"./数据/{class_name}_dashboard.html", "w", encoding="utf-8") as f:
        f.write("<html><head><title>班级数据看板</title></head><body><h1>班级数据看板</h1>")
        f.write("<table style='width:100%'>")
        
        for i in range(0, len(charts), 2):
            f.write("<tr>")
            f.write("<td style='width:50%'>")
            f.write(charts[i].render_embed())  # 渲染第一个图表
            f.write("</td>")
            
            if i + 1 < len(charts):
                f.write("<td style='width:50%'>")
                f.write(charts[i + 1].render_embed())  # 渲染第二个图表
                f.write("</td>")
            f.write("</tr>")
        
        f.write("</table></body></html>")
    print(f"{class_name}的看板已生成为 ./数据/{class_name}_dashboard.html")

# 创建主页面用于选择班级
with open("all_classes_dashboard.html", "w", encoding="utf-8") as f:
    f.write("""
    <html>
    <head>
        <title>所有班级数据看板</title>
        <script>
            function showClassBoard(className) {
                const iframe = document.getElementById("classIframe");
                iframe.src = "./数据/" + className + "_dashboard.html";
            }
        </script>
    </head>
    <body>
        <h1>所有班级数据看板</h1>
        <label for="classSelect">选择班级:</label>
        <select id="classSelect" onchange="showClassBoard(this.value)">
    """)
    
    for i, file_path in enumerate(file_paths):
        class_name = os.path.splitext(os.path.basename(file_path))[0]
        selected = "selected" if i == 0 else ""
        f.write(f'<option value="{class_name}" {selected}>{class_name}</option>')
    
    f.write("""
        </select>
        <iframe id="classIframe" src="./数据/SubmitRecord-Class1_dashboard.html" style="width: 100%; height: 800px; border: none;"></iframe>
    </body>
    </html>
    """)
print("所有班级的主页面已生成")


SubmitRecord-Class1的看板已生成为 ./数据/SubmitRecord-Class1_dashboard.html
SubmitRecord-Class2的看板已生成为 ./数据/SubmitRecord-Class2_dashboard.html
SubmitRecord-Class3的看板已生成为 ./数据/SubmitRecord-Class3_dashboard.html
SubmitRecord-Class4的看板已生成为 ./数据/SubmitRecord-Class4_dashboard.html
SubmitRecord-Class5的看板已生成为 ./数据/SubmitRecord-Class5_dashboard.html
SubmitRecord-Class6的看板已生成为 ./数据/SubmitRecord-Class6_dashboard.html
SubmitRecord-Class7的看板已生成为 ./数据/SubmitRecord-Class7_dashboard.html
SubmitRecord-Class8的看板已生成为 ./数据/SubmitRecord-Class8_dashboard.html
SubmitRecord-Class9的看板已生成为 ./数据/SubmitRecord-Class9_dashboard.html
SubmitRecord-Class10的看板已生成为 ./数据/SubmitRecord-Class10_dashboard.html
SubmitRecord-Class11的看板已生成为 ./数据/SubmitRecord-Class11_dashboard.html
SubmitRecord-Class12的看板已生成为 ./数据/SubmitRecord-Class12_dashboard.html
SubmitRecord-Class13的看板已生成为 ./数据/SubmitRecord-Class13_dashboard.html
SubmitRecord-Class14的看板已生成为 ./数据/SubmitRecord-Class14_dashboard.html
SubmitRecord-Class15的看板已生成为 ./数据/SubmitRecord-Class1